# NDAWN 30 Day Weather Report

## Import Data

### Importing Packages

In [100]:
import requests
import pandas as pd
import os

### Pulling in Weather Data

In [343]:
ndawn_base = r'https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=174&station=142&station=138&station=161&station=9&station=10&station=118&station=56&station=11&station=12&station=58&station=13&station=84&station=55&station=7&station=87&station=14&station=15&station=96&station=16&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=19&station=129&station=20&station=101&station=81&station=21&station=97&station=22&station=75&station=2&station=172&station=139&station=23&station=62&station=86&station=24&station=89&station=126&station=93&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=70&station=127&station=27&station=132&station=28&station=29&station=30&station=31&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=105&station=82&station=34&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=71&station=37&station=38&station=39&station=130&station=73&station=40&station=41&station=54&station=69&station=113&station=128&station=42&station=43&station=103&station=116&station=88&station=114&station=3&station=163&station=64&station=115&station=67&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=74&station=60&station=125&station=8&station=47&station=122&station=108&station=5&station=152&station=48&station=68&station=49&station=50&station=91&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=57&station=149&station=148&station=110&variable=ddmxt&variable=ddmnt&variable=ddavt&ttype=daily&quick_pick=30_d'

In [344]:
rqst_objct = requests.get(ndawn_base)

with open('AvgTemp_AllStations.csv', 'w') as newCSV_txt:
    newCSV_txt.write(rqst_objct.content.decode('utf-8'))

In [345]:
# reading the CSV and visually checking it looks right 

avg_temp_df = pd.read_csv('AvgTemp_AllStations.csv', skiprows = 5)
avg_temp_df

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Max Temp,Max Temp Flag,Min Temp,Min Temp Flag,Avg Temp,Avg Temp Flag
0,NaN,deg,deg,ft,NaN,NaN,NaN,Degrees F,NaN,Degrees F,NaN,Degrees F,NaN
1,Ada,47.32119,-96.51406,910,2022.0,10.0,29.0,60.584,NaN,30.619,NaN,45.602,NaN
2,Ada,47.32119,-96.51406,910,2022.0,10.0,30.0,58.784,NaN,22.091,NaN,40.438,NaN
3,Ada,47.32119,-96.51406,910,2022.0,10.0,31.0,64.022,NaN,35.074,NaN,49.548,NaN
4,Ada,47.32119,-96.51406,910,2022.0,11.0,1.0,70.610,NaN,27.538,NaN,49.074,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4226,Zeeland,46.01351,-99.68768,2070,2022.0,11.0,23.0,37.144,NaN,15.116,NaN,26.130,NaN
4227,Zeeland,46.01351,-99.68768,2070,2022.0,11.0,24.0,33.303,NaN,15.440,NaN,24.372,NaN
4228,Zeeland,46.01351,-99.68768,2070,2022.0,11.0,25.0,42.800,NaN,14.900,NaN,28.850,NaN
4229,Zeeland,46.01351,-99.68768,2070,2022.0,11.0,26.0,38.952,NaN,22.595,NaN,30.774,NaN


In [346]:
# Cleaning up the dataset by removing the top row and the columns with no values

avg_temp = avg_temp_df.iloc[1:]
base = avg_temp.drop(columns = ['Max Temp Flag', 'Min Temp Flag', 'Avg Temp Flag', 'Elevation', 'Year', 'Month', 'Day'])

In [347]:
# group the data by station and find the max/min/avg temp

min_temps = base.drop(columns = ['Max Temp', 'Avg Temp'])
min_temps['Min Temp'] = min_temps['Min Temp'].astype(float)
min_temps = min_temps.groupby(['Station Name']).min()

max_temps = base.drop(columns = ['Min Temp', 'Avg Temp'])
max_temps['Max Temp'] = max_temps['Max Temp'].astype(float)
max_temps = max_temps.groupby(['Station Name']).max()

avg_temps = base.drop(columns = ['Min Temp', 'Max Temp'])
avg_temps['Avg Temp'] = avg_temps['Avg Temp'].astype(float)
avg_temps = avg_temps.groupby(['Station Name']).mean()

In [348]:
min_max = min_temps.merge(max_temps, how='inner', on=('Station Name'))
min_max = min_max.merge(avg_temps, how='inner', on=('Station Name'))
min_max = min_max.drop(columns = ['Latitude_y', 'Longitude_y'])
min_max = min_max.rename(columns = {'Latitude_x':'Latitude', 'Longitude_x':'Longitude'})

min_max

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

,Latitude,Longitude,Min Temp,Max Temp,Avg Temp
Station Name,,,,,
Ada,47.32119,-96.51406,2.534,70.610,28.875800
Adams,48.49988,-98.07588,-2.740,67.280,24.684367
Alamo,48.54652,-103.47186,-12.982,63.842,22.120733
Alvarado,48.245942,-97.021532,0.302,68.918,27.125367
Amidon,46.48844,-103.31629,-16.006,72.230,26.382900
...,...,...,...,...,...
Williston,48.13308,-103.73859,-8.338,65.876,24.412900
Wishek,46.26033,-99.66585,-5.368,74.408,26.459600
Wolford,48.51603,-99.62441,-6.772,70.250,23.559967


In [338]:
min_max.to_csv('min_max_avg.csv', index=False)

In [328]:
station_name_base = base.drop(columns = ['Max Temp', 'Min Temp', 'Avg Temp'])
station_name_base.to_csv('station_name_base.csv', index=False)

In [ ]:
# joining the 2 datasets together

arcpy.management.AddJoin("min_max_avg.csv", "Latitude", "station_name_base.csv", "Latitude", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")

In [ ]:
arcpy.conversion.TableToTable("min_max_avg.csv", r"C:\Users\tande\Documents\ArcGIS\Projects\GIS5571_Lab3Part2\GIS5571_Lab3Part2.gdb", "full_dataset", '', 'Latitude "Latitude" true true false 8 Double 0 0,First,#,min_max_avg.csv,min_max_avg.csv.Latitude,-1,-1;Longitude "Longitude" true true false 8 Double 0 0,First,#,min_max_avg.csv,min_max_avg.csv.Longitude,-1,-1;Min Temp "Min Temp" true true false 8 Double 0 0,First,#,min_max_avg.csv,min_max_avg.csv.Min Temp,-1,-1;Max Temp "Max Temp" true true false 8 Double 0 0,First,#,min_max_avg.csv,min_max_avg.csv.Max Temp,-1,-1;Avg Temp "Avg Temp" true true false 8 Double 0 0,First,#,min_max_avg.csv,min_max_avg.csv.Avg Temp,-1,-1;Station Name "Station Name" true true false 8000 Text 0 0,First,#,min_max_avg.csv,min_max_avg.csv.Station Name,0,8000;Station Name "Station Name" true true false 8000 Text 0 0,First,#,min_max_avg.csv,station_name_base.csv.Station Name,0,8000;Latitude "Latitude" true true false 8 Double 0 0,First,#,min_max_avg.csv,station_name_base.csv.Latitude,-1,-1;Longitude "Longitude" true true false 8 Double 0 0,First,#,min_max_avg.csv,station_name_base.csv.Longitude,-1,-1', '')

In [ ]:
# cleaned up the dataset to remove duplicate columns (lat, long, station name)

## Plotting all Weather Station Locations for Visual Confirmation

In [215]:
# Converting lat/long to XY coords

arcpy.management.XYTableToPoint("full_dataset", r"C:\Users\tande\Documents\ArcGIS\Projects\GIS5571_Lab3Part2\GIS5571_Lab3Part2.gdb\full_dataset_XYTableToPoint", "Longitude", "Latitude", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

<Result 'C:\\Users\\tande\\Documents\\ArcGIS\\Projects\\GIS5571_Lab3Part2\\GIS5571_Lab3Part2.gdb\\min_max_XYTableToPoint'>

## IDW

### Average Temps

In [ ]:
# Power 10

arcpy.ga.IDW("full_dataset_XYTableToPoint", "Avg_Temp", "idw_output_avg_power10", None, 0.0144790399999999, 10, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "Avg_Temp")

# Power 5

arcpy.ga.IDW("full_dataset_XYTableToPoint", "Avg_Temp", "idw_output_avg_power5", None, 0.0144790399999999, 5, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "Avg_Temp")

# Power 2

arcpy.ga.IDW("full_dataset_XYTableToPoint", "Avg_Temp", "idw_output_avg_power2", None, 0.0144790399999999, 2, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "Avg_Temp")

### Max Temps

In [ ]:
# Power 10

arcpy.ga.IDW("full_dataset_XYTableToPoint", "Max_Temp", "idw_output_max_power10", None, 0.0144790399999999, 10, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "Max_Temp")

# Power 5

arcpy.ga.IDW("full_dataset_XYTableToPoint", "Max_Temp", "idw_output_max_power5", None, 0.0144790399999999, 5, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "Max_Temp")

# Power 2

arcpy.ga.IDW("full_dataset_XYTableToPoint", "Max_Temp", "idw_output_max_power2", None, 0.0144790399999999, 2, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "Max_Temp")

### Min Temps

In [ ]:
# Cannot have negative values to calculate IDW

# Finding the absolute value of the min temps to eliminate negative values

arcpy.management.CalculateField("full_dataset_XYTableToPoint", "New Zero", "Abs($feature.Min_Temp)", "ARCADE", '', "FLOAT", "NO_ENFORCE_DOMAINS")

# Finding the inverse of the absolute values so they are more

arcpy.management.CalculateField("full_dataset_XYTableToPoint", "MinTempZero", "1 / $feature.MinTempZero", "ARCADE", '', "TEXT", "NO_ENFORCE_DOMAINS")

# Power 10

arcpy.ga.IDW("full_dataset_XYTableToPoint", "MinTempZero", "idw_output_min_power10", None, 0.0144790399999999, 10, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "MinTempZero")

# Power 5

arcpy.ga.IDW("full_dataset_XYTableToPoint", "MinTempZero", "idw_output_min_power5", None, 0.0144790399999999, 5, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "MinTempZero")

# Power 2

arcpy.ga.IDW("full_dataset_XYTableToPoint", "MinTempZero", "idw_output_min_power2", None, 0.0144790399999999, 2, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "MinTempZero")

## Kriging - Empirical Bayesian Kriging

### Average Temps

In [ ]:
# Power 

arcpy.ga.EmpiricalBayesianKriging("full_dataset_XYTableToPoint", "Avg_Temp", "Kriging_emp_avg_power", None, 0.0144790399999999, "NONE", 100, 1, 100, "NBRTYPE=StandardCircular RADIUS=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "PREDICTION", 0.5, "EXCEED", None, "POWER")

# Linear

arcpy.ga.EmpiricalBayesianKriging("full_dataset_XYTableToPoint", "Avg_Temp", "Kriging_emp_avg_linear", None, 0.0144790399999999, "NONE", 100, 1, 100, "NBRTYPE=StandardCircular RADIUS=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "PREDICTION", 0.5, "EXCEED", None, "LINEAR")

# Spline

arcpy.ga.EmpiricalBayesianKriging("full_dataset_XYTableToPoint", "Avg_Temp", "Kriging_emp_avg_spline", None, 0.0144790399999999, "NONE", 100, 1, 100, "NBRTYPE=StandardCircular RADIUS=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "PREDICTION", 0.5, "EXCEED", None, "THIN_PLATE_SPLINE")

### Min Temps

In [ ]:
# Power

arcpy.ga.EmpiricalBayesianKriging("full_dataset_XYTableToPoint", "Min_Temp", "Kriging_emp_min_power", None, 0.0144790399999999, "NONE", 100, 1, 100, "NBRTYPE=StandardCircular RADIUS=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "PREDICTION", 0.5, "EXCEED", None, "POWER")

# Linear

arcpy.ga.EmpiricalBayesianKriging("full_dataset_XYTableToPoint", "Min_Temp", "Kriging_emp_min_linear", None, 0.0144790399999999, "NONE", 100, 1, 100, "NBRTYPE=StandardCircular RADIUS=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "PREDICTION", 0.5, "EXCEED", None, "LINEAR")

# Spline

arcpy.ga.EmpiricalBayesianKriging("full_dataset_XYTableToPoint", "Min_Temp", "Kriging_emp_min_spline", None, 0.0144790399999999, "NONE", 100, 1, 100, "NBRTYPE=StandardCircular RADIUS=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "PREDICTION", 0.5, "EXCEED", None, "THIN_PLATE_SPLINE")

### Max Temps

In [ ]:
# Power

arcpy.ga.EmpiricalBayesianKriging("full_dataset_XYTableToPoint", "Max_Temp", "Kriging_emp_max_power", None, 0.0144790399999999, "NONE", 100, 1, 100, "NBRTYPE=StandardCircular RADIUS=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "PREDICTION", 0.5, "EXCEED", None, "POWER")

# Linear

arcpy.ga.EmpiricalBayesianKriging("full_dataset_XYTableToPoint", "Max_Temp", "Kriging_emp_max_linear", None, 0.0144790399999999, "NONE", 100, 1, 100, "NBRTYPE=StandardCircular RADIUS=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "PREDICTION", 0.5, "EXCEED", None, "LINEAR")

# Spline

arcpy.ga.EmpiricalBayesianKriging("full_dataset_XYTableToPoint", "Max_Temp", "Kriging_emp_max_spline", None, 0.0144790399999999, "NONE", 100, 1, 100, "NBRTYPE=StandardCircular RADIUS=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "PREDICTION", 0.5, "EXCEED", None, "THIN_PLATE_SPLINE")

## Local Polynomial Interpolation

### Average Temps

In [ ]:
# Exponential

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Avg_Temp", "LPI_avg_order1_epx", None, 0.0144790399999999, 1, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "EXPONENTIAL", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

# Gaussian

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Avg_Temp", "LPI_avg_order1_gaus", None, 0.0144790399999999, 1, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "GAUSSIAN", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

# Quartic

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Avg_Temp", "LPI_avg_order1_quar", None, 0.0144790399999999, 1, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "QUARTIC", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

# Epan

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Avg_Temp", "LPI_avg_order1_epan", None, 0.0144790399999999, 1, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "EPANECHNIKOV", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

# Fifth-order polynomial

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Avg_Temp", "LPI_avg_order1_fifth", None, 0.0144790399999999, 1, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "POLYNOMIAL5", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

# Constant

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Avg_Temp", "LPI_avg_order1_constant", None, 0.0144790399999999, 1, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "CONSTANT", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

### Min Temps

In [ ]:
# Quartic

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Min_Temp", "LPI_min_order4_quar", None, 0.0144790399999999, 2, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "QUARTIC", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

# Exponential

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Min_Temp", "LPI_min_order1_exp", None, 0.0144790399999999, 2, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "EXPONENTIAL", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

# Gaussian

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Min_Temp", "LPI_min_order1_gaus", None, 0.0144790399999999, 2, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "GAUSSIAN", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

# Epan

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Min_Temp", "LPI_min_order1_epan", None, 0.0144790399999999, 2, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "EPANECHNIKOV", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

# Fifth-order polynomial

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Min_Temp", "LPI_min_order1_fifth", None, 0.0144790399999999, 2, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "POLYNOMIAL5", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

# Constant

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Min_Temp", "LPI_min_order1_constant", None, 0.0144790399999999, 2, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "CONSTANT", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

### Max Temps

In [ ]:
# Constant

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Max_Temp", "LPI_max_order1_constant", None, 0.0144790399999999, 2, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "CONSTANT", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

# Exponential

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Max_Temp", "LPI_max_order1_exp", None, 0.0144790399999999, 2, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "EXPONENTIAL", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

# Gaussian

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Max_Temp", "LPI_max_order1_gaus", None, 0.0144790399999999, 2, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "GAUSSIAN", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

# Quartic

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Max_Temp", "LPI_max_order1_quar", None, 0.0144790399999999, 2, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "QUARTIC", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

# Epan

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Max_Temp", "LPI_max_order1_epan", None, 0.0144790399999999, 2, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "EPANECHNIKOV", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

# Fifth-order polynomial

arcpy.ga.LocalPolynomialInterpolation("full_dataset_XYTableToPoint", "Max_Temp", "LPI_max_order1_epan", None, 0.0144790399999999, 2, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "POLYNOMIAL5", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")


## Hot/Cold Spots for Result Verification

In [ ]:
# Min Temps

arcpy.stats.HotSpots("full_dataset_XYTableToPoint", "Min_Temp", r"C:\Users\tande\Documents\ArcGIS\Projects\GIS5571_Lab3Part2\GIS5571_Lab3Part2.gdb\full_dataset_XYTableToPoint_HotSpots", "K_NEAREST_NEIGHBORS", "EUCLIDEAN_DISTANCE", "ROW", None, None, None, "NO_FDR", 8)

# Max Temps

arcpy.stats.HotSpots("full_dataset_XYTableToPoint", "Max_Temp", r"C:\Users\tande\Documents\ArcGIS\Projects\GIS5571_Lab3Part2\GIS5571_Lab3Part2.gdb\full_dataset_XYTableToPoint_MaxHotSpots", "K_NEAREST_NEIGHBORS", "EUCLIDEAN_DISTANCE", "ROW", None, None, None, "NO_FDR", 10)

# Avg Temps

arcpy.stats.HotSpots("full_dataset_XYTableToPoint", "Avg_Temp", r"C:\Users\tande\Documents\ArcGIS\Projects\GIS5571_Lab3Part2\GIS5571_Lab3Part2.gdb\full_dataset_XYTableToPoint_AvgHotSpots", "K_NEAREST_NEIGHBORS", "EUCLIDEAN_DISTANCE", "ROW", None, None, None, "NO_FDR", 10)